# Data preparation

In [ ]:
from pathlib import Path

_ = """
path: /home/robo/resciu-dataset/files # dataset root dir
train: train/images # train images (relative to 'path')
val: evaluation_dataset/images # val images (relative to 'path')

# Classes
names:
  0: people
"""

data_path = Path("aerovision.yaml")

with open(data_path, "w") as file:
    file.write(_)

# Training

In [ ]:
from datetime import datetime
from ultralytics import YOLO
from wandb.integration.ultralytics import add_wandb_callback

today = datetime.now().strftime("%Y-%m-%d %H:%M")

model = YOLO(model="yolo11m.pt", task="detect")

add_wandb_callback(
    model,
    enable_model_checkpointing=True,
    enable_train_validation_logging=False,
    enable_validation_logging=False,
    enable_prediction_logging=False,
);

In [ ]:
import wandb

wandb.login()

In [ ]:
with wandb.init(
        entity="rescIU",
        project="PART1",
        name=f"yolo11 {today}",
        notes="YOLO11m on Aerovision, disable mosaic",
        job_type="training",
        tags=["aerovision", "yolo11m"],
        reinit=True,
) as run:
    # https://docs.ultralytics.com/modes/train/#train-settings
    results = model.train(
        project=run.project,
        name=run.name,
        data="aerovision.yaml",
        batch=25,
        cos_lr=True,
        mosaic=0.0,
        flipud=0.5,
        fliplr=0.5,
        epochs=400,
    )

# Evaluation

In [ ]:
import pandas as pd


# Function to run validation for a specific image size and return metrics
def get_val_metrics(img_size, batch=8):
    _val_results = model.val(data="aerovision.yaml", batch=batch, imgsz=img_size, verbose=False)
    _metrics = {
        "Inference Size": f"{size}px",
        # "P": _val_results.box.map50,
        # "R": _val_results.box.map,
        "mAP50-95": _val_results.box.map,
        "mAP50": _val_results.box.map50,
        "Preprocess Time (ms)": _val_results.speed["preprocess"],  # Preprocessing time
        "Inference Time (ms)": _val_results.speed["inference"],  # Inference time
        "Postprocess Time (ms)": _val_results.speed["postprocess"]  # Postprocessing time
    }
    return _metrics


# List of image sizes to test
image_sizes = [640, 1280, 2016]

# Collect metrics for each image size
val_results = []
for size in image_sizes:
    metrics = get_val_metrics(size)
    val_results.append(metrics)

# Create DataFrame from results
df = pd.DataFrame(val_results)
df